In [2]:
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from scipy.misc import imresize
import os

In [8]:
labels_path = os.getcwd() + "\\" + os.path.join( "data", "labels.csv", "")

In [9]:
labels_path

'C:\\Users\\Vladamir Mazin\\DeepNN\\data\\labels.csv\\'

In [10]:
data_labels = pd.read_csv(labels_path + "labels.csv")

In [11]:
type(data_labels)

pandas.core.frame.DataFrame

In [12]:
df = pd.DataFrame(np.random.randn(100, 2))
# df

In [13]:
def get_train_test(data_labels):
    msk = np.random.rand(len(data_labels)) < 0.8
    train = data_labels[msk]
    test = data_labels[~msk]
    return train, test

In [14]:
train, test = get_train_test(data_labels)

In [15]:
# train['id']

In [16]:
img_path = os.getcwd() + "\\" + os.path.join( "data", "train", "train", "")

In [17]:
# from itertools import islice
# for x in islice(train.iterrows(), 10):
#     print(x[1]['id'])

In [18]:
# preparing the train dataset
train_img = []
for row in train.iterrows():
    temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
    temp_img = image.img_to_array(temp_img)
    train_img.append(temp_img)
    
#converting train images to array and applying mean subtraction processing
train_img = np.array(train_img) 
train_img = preprocess_input(train_img)

In [19]:
# applying the same procedure with the test dataset
test_img=[]
for row in test.iterrows():
    temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
    temp_img = image.img_to_array(temp_img)
    test_img.append(temp_img)

test_img=np.array(test_img) 
test_img=preprocess_input(test_img)

In [ ]:
from keras.models import Model

def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    model = VGG16(weights='imagenet', include_top=True)

    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-1].outbound_nodes = []

          x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)

#To set the first 8 layers to non-trainable (weights will not be updated)

          for layer in model.layers[:8]:

       layer.trainable = False

# Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
train_y=np.asarray(train['breed'])

le = LabelEncoder()

train_y = le.fit_transform(train_y)

train_y=to_categorical(train_y)

train_y=np.array(train_y)

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.2, random_state=42)

In [ ]:
# Example to fine-tune on 3000 samples from Cifar10

img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 10 
batch_size = 16 
nb_epoch = 10

# Load our model
model = vgg16_model(img_rows, img_cols, channel, num_classes)

model.summary()
# Start Fine-tuning
model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

# Cross-entropy loss score
score = log_loss(Y_valid, predictions_valid)